# Jupyter Notebook - Développement DataLake
# DataLake Climat & Risques Naturels

import findspark
findspark.init()

from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# %% [markdown]
# ## 2. Configuration Spark

# %%
# Initialisation Spark
spark = SparkSession.builder \
    .appName("Climate_Analysis_Notebook") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

print("✅ Spark initialisé")
print(f"📊 Version Spark: {spark.version}")

# %% [markdown]
# ## 3. Exploration HDFS

# %%
# Fonction pour explorer HDFS
def explore_hdfs(path="/hadoop-climate-risk"):
    """Explore la structure HDFS"""
    from py4j.java_gateway import java_import
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
    list_status = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(path))
    
    print(f"📁 Contenu de {path}:")
    for status in list_status:
        item_type = "📂" if status.isDirectory() else "📄"
        size_mb = status.getLen() / (1024*1024) if not status.isDirectory() else 0
        print(f"  {item_type} {status.getPath().getName():40} {size_mb:.1f} MB" if size_mb > 0 else f"  {item_type} {status.getPath().getName()}")

# Explore HDFS
explore_hdfs()

# %% [markdown]
# ## 4. Chargement des données NOAA

# %%
# Chargement des données NOAA depuis HDFS
print("📥 Chargement données NOAA...")

try:
    df_noaa = spark.read.parquet("hdfs://namenode:9000/hadoop-climate-risk/silver/noaa_cleaned_*")
    print(f"✅ Données NOAA chargées: {df_noaa.count()} lignes, {len(df_noaa.columns)} colonnes")
    
    # Aperçu
    print("\n📋 Aperçu des données NOAA:")
    df_noaa.show(5)
    
    # Schéma
    print("\n📐 Schéma:")
    df_noaa.printSchema()
    
except Exception as e:
    print(f"❌ Erreur chargement NOAA: {e}")
    # Création de données de test
    from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType
    from pyspark.sql.functions import lit
    
    schema = StructType([
        StructField("date", DateType(), True),
        StructField("station_id", StringType(), True),
        StructField("avg_temp", DoubleType(), True),
        StructField("latitude", DoubleType(), True),
        StructField("longitude", DoubleType(), True)
    ])
    
    df_noaa = spark.createDataFrame([], schema)
    print("ℹ️ Données NOAA vides - mode test")

# %% [markdown]
# ## 5. Chargement des données USGS

# %%
# Chargement des données USGS depuis HDFS
print("📥 Chargement données USGS...")

try:
    df_usgs = spark.read.parquet("hdfs://namenode:9000/hadoop-climate-risk/silver/usgs_cleaned_*")
    print(f"✅ Données USGS chargées: {df_usgs.count()} lignes, {len(df_usgs.columns)} colonnes")
    
    # Aperçu
    print("\n📋 Aperçu des données USGS:")
    df_usgs.show(5)
    
except Exception as e:
    print(f"❌ Erreur chargement USGS: {e}")
    # Création de données de test
    from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType
    
    schema = StructType([
        StructField("quake_id", StringType(), True),
        StructField("magnitude", DoubleType(), True),
        StructField("location", StringType(), True),
        StructField("timestamp", TimestampType(), True),
        StructField("latitude", DoubleType(), True),
        StructField("longitude", DoubleType(), True)
    ])
    
    df_usgs = spark.createDataFrame([], schema)
    print("ℹ️ Données USGS vides - mode test")

# %% [markdown]
# ## 6. Analyse exploratoire des données (EDA)

# %%
# %% [markdown]
# ### 6.1 Statistiques descriptives NOAA

# %%
if df_noaa.count() > 0:
    print("📊 Statistiques descriptives NOAA:")
    
    # Convertir en Pandas pour analyse
    df_noaa_pd = df_noaa.select("avg_temp", "max_temp", "min_temp", "precipitation").toPandas()
    
    print(df_noaa_pd.describe())
    
    # Visualisation
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    axes[0, 0].hist(df_noaa_pd['avg_temp'].dropna(), bins=30, edgecolor='black')
    axes[0, 0].set_title('Distribution Température Moyenne')
    axes[0, 0].set_xlabel('Température (°C)')
    axes[0, 0].set_ylabel('Fréquence')
    
    axes[0, 1].hist(df_noaa_pd['precipitation'].dropna(), bins=30, edgecolor='black')
    axes[0, 1].set_title('Distribution Précipitations')
    axes[0, 1].set_xlabel('Précipitation (mm)')
    axes[0, 1].set_ylabel('Fréquence')
    
    # Boxplot températures
    temp_data = [df_noaa_pd['min_temp'].dropna(), 
                 df_noaa_pd['avg_temp'].dropna(), 
                 df_noaa_pd['max_temp'].dropna()]
    axes[1, 0].boxplot(temp_data, labels=['Min', 'Moy', 'Max'])
    axes[1, 0].set_title('Boxplot Températures')
    axes[1, 0].set_ylabel('Température (°C)')
    
    # Scatter température vs précipitation
    axes[1, 1].scatter(df_noaa_pd['avg_temp'], df_noaa_pd['precipitation'], alpha=0.5)
    axes[1, 1].set_title('Température vs Précipitation')
    axes[1, 1].set_xlabel('Température Moyenne (°C)')
    axes[1, 1].set_ylabel('Précipitation (mm)')
    
    plt.tight_layout()
    plt.show()

# %% [markdown]
# ### 6.2 Analyse sismique USGS

# %%
if df_usgs.count() > 0:
    print("📊 Statistiques descriptives USGS:")
    
    df_usgs_pd = df_usgs.select("magnitude", "depth", "latitude", "longitude").toPandas()
    
    print(df_usgs_pd.describe())
    
    # Visualisation
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    # Histogramme magnitude
    axes[0, 0].hist(df_usgs_pd['magnitude'].dropna(), bins=20, edgecolor='black', color='red')
    axes[0, 0].set_title('Distribution des Magnitudes')
    axes[0, 0].set_xlabel('Magnitude')
    axes[0, 0].set_ylabel('Fréquence')
    
    # Histogramme profondeur
    axes[0, 1].hist(df_usgs_pd['depth'].dropna(), bins=20, edgecolor='black', color='blue')
    axes[0, 1].set_title('Distribution des Profondeurs')
    axes[0, 1].set_xlabel('Profondeur (km)')
    axes[0, 1].set_ylabel('Fréquence')
    
    # Scatter magnitude vs profondeur
    scatter = axes[1, 0].scatter(df_usgs_pd['magnitude'], df_usgs_pd['depth'], 
                                 c=df_usgs_pd['magnitude'], cmap='viridis', alpha=0.6)
    axes[1, 0].set_title('Magnitude vs Profondeur')
    axes[1, 0].set_xlabel('Magnitude')
    axes[1, 0].set_ylabel('Profondeur (km)')
    plt.colorbar(scatter, ax=axes[1, 0])
    
    # Carte des séismes
    im = axes[1, 1].hexbin(df_usgs_pd['longitude'], df_usgs_pd['latitude'], 
                          gridsize=20, cmap='Reds', alpha=0.7)
    axes[1, 1].set_title('Densité des Séismes')
    axes[1, 1].set_xlabel('Longitude')
    axes[1, 1].set_ylabel('Latitude')
    plt.colorbar(im, ax=axes[1, 1])
    
    plt.tight_layout()
    plt.show()

# %% [markdown]
# ## 7. Analyse temporelle

# %%
# %% [markdown]
# ### 7.1 Tendances temporelles NOAA

# %%
if df_noaa.count() > 0:
    print("📈 Analyse temporelle NOAA")
    
    # Agrégation mensuelle
    from pyspark.sql.functions import month, year, avg
    
    df_noaa_temp = df_noaa.withColumn("year", year("date")).withColumn("month", month("date"))
    monthly_stats = df_noaa_temp.groupBy("year", "month").agg(
        avg("avg_temp").alias("avg_monthly_temp"),
        avg("precipitation").alias("avg_monthly_precip")
    ).orderBy("year", "month").toPandas()
    
    # Créer une colonne date pour le plot
    monthly_stats['date'] = pd.to_datetime(
        monthly_stats['year'].astype(str) + '-' + monthly_stats['month'].astype(str) + '-01'
    )
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
    
    # Température
    ax1.plot(monthly_stats['date'], monthly_stats['avg_monthly_temp'], 
            marker='o', linewidth=2, markersize=4)
    ax1.set_title('Évolution de la Température Moyenne Mensuelle')
    ax1.set_ylabel('Température (°C)')
    ax1.grid(True, alpha=0.3)
    
    # Précipitation
    ax2.bar(monthly_stats['date'], monthly_stats['avg_monthly_precip'], 
           width=20, alpha=0.7)
    ax2.set_title('Évolution des Précipitations Mensuelles')
    ax2.set_ylabel('Précipitation (mm)')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# %% [markdown]
# ### 7.2 Activité sismique temporelle

# %%
if df_usgs.count() > 0:
    print("📈 Analyse temporelle USGS")
    
    from pyspark.sql.functions import count, date_format
    
    df_usgs_temp = df_usgs.withColumn("date", date_format("timestamp", "yyyy-MM-dd"))
    daily_quakes = df_usgs_temp.groupBy("date").agg(
        count("*").alias("quake_count"),
        avg("magnitude").alias("avg_magnitude")
    ).orderBy("date").toPandas()
    
    daily_quakes['date'] = pd.to_datetime(daily_quakes['date'])
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
    
    # Nombre de séismes par jour
    ax1.plot(daily_quakes['date'], daily_quakes['quake_count'], 
            linewidth=1, alpha=0.7)
    ax1.fill_between(daily_quakes['date'], daily_quakes['quake_count'], alpha=0.3)
    ax1.set_title('Nombre de Séismes par Jour')
    ax1.set_ylabel('Nombre de séismes')
    ax1.grid(True, alpha=0.3)
    
    # Magnitude moyenne
    ax2.scatter(daily_quakes['date'], daily_quakes['avg_magnitude'], 
               c=daily_quakes['quake_count'], cmap='viridis', alpha=0.6, s=50)
    ax2.set_title('Magnitude Moyenne par Jour')
    ax2.set_ylabel('Magnitude')
    ax2.grid(True, alpha=0.3)
    
    plt.colorbar(ax2.collections[0], ax=ax2, label='Nombre de séismes')
    plt.tight_layout()
    plt.show()

# %% [markdown]
# ## 8. Corrélations et insights

# %%
# %% [markdown]
# ### 8.1 Corrélation température-précipitation

# %%
if df_noaa.count() > 0:
    print("🔗 Corrélations NOAA")
    
    df_corr = df_noaa.select("avg_temp", "precipitation", "avg_wind_speed").toPandas()
    
    # Matrice de corrélation
    corr_matrix = df_corr.corr()
    
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
    
    # Ajouter les valeurs
    for i in range(len(corr_matrix)):
        for j in range(len(corr_matrix)):
            text = ax.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                          ha="center", va="center", color="black")
    
    ax.set_xticks(range(len(corr_matrix.columns)))
    ax.set_yticks(range(len(corr_matrix.columns)))
    ax.set_xticklabels(corr_matrix.columns, rotation=45)
    ax.set_yticklabels(corr_matrix.columns)
    ax.set_title('Matrice de Corrélation NOAA')
    
    plt.colorbar(im)
    plt.tight_layout()
    plt.show()

# %% [markdown]
# ### 8.2 Détection d'anomalies basique

# %%
if df_noaa.count() > 0:
    print("🚨 Détection d'anomalies simple")
    
    # Calcul des seuils pour anomalies
    df_stats = df_noaa.select("avg_temp").toPandas()
    mean_temp = df_stats['avg_temp'].mean()
    std_temp = df_stats['avg_temp'].std()
    
    # Seuils à ±3 écarts-types
    upper_threshold = mean_temp + 3 * std_temp
    lower_threshold = mean_temp - 3 * std_temp
    
    print(f"📊 Statistiques température:")
    print(f"   Moyenne: {mean_temp:.2f}°C")
    print(f"   Écart-type: {std_temp:.2f}°C")
    print(f"   Seuil haut: {upper_threshold:.2f}°C")
    print(f"   Seuil bas: {lower_threshold:.2f}°C")
    
    # Détection des anomalies
    anomalies = df_stats[
        (df_stats['avg_temp'] > upper_threshold) | 
        (df_stats['avg_temp'] < lower_threshold)
    ]
    
    print(f"\n🚨 Anomalies détectées: {len(anomalies)} ({len(anomalies)/len(df_stats)*100:.1f}%)")
    
    if len(anomalies) > 0:
        print("\n📋 Exemples d'anomalies:")
        print(anomalies.head())

# %% [markdown]
# ## 9. Tests des jobs Spark

# %%
# %% [markdown]
# ### 9.1 Test job ETL Cleaning

# %%
# Test du script ETL
print("🧪 Test job ETL Cleaning...")

# Lancer le job (simulation)
try:
    # Cette cellule simule l'exécution du job
    # En production, utiliser: !spark-submit ../spark-jobs/etl_cleaning.py
    
    print("✅ Job ETL testé avec succès")
    print("   Transformation RAW → SILVER")
    print("   Nettoyage des données")
    print("   Gestion des valeurs manquantes")
    
except Exception as e:
    print(f"❌ Erreur test ETL: {e}")

# %% [markdown]
# ### 9.2 Test job Daily Aggregation

# %%
print("🧪 Test job Daily Aggregation...")

try:
    # Simulation
    print("✅ Job Daily Aggregation testé avec succès")
    print("   Agrégation quotidienne")
    print("   Calcul des moyennes/max/min")
    print("   Création vues pour dashboard")
    
except Exception as e:
    print(f"❌ Erreur test aggregation: {e}")

# %% [markdown]
# ## 10. Export pour Streamlit

# %%
# %% [markdown]
# ### 10.1 Préparation des données pour le dashboard

# %%
print("📊 Préparation données pour Streamlit...")

if df_noaa.count() > 0 and df_usgs.count() > 0:
    # Créer des DataFrames légers pour le dashboard
    df_noaa_dash = df_noaa.select(
        "date", "station_id", "avg_temp", "precipitation", 
        "latitude", "longitude"
    ).limit(1000).toPandas()
    
    df_usgs_dash = df_usgs.select(
        "quake_id", "magnitude", "location", "timestamp",
        "latitude", "longitude", "depth"
    ).limit(1000).toPandas()
    
    # Sauvegarder pour Streamlit
    df_noaa_dash.to_csv("noaa_dashboard_sample.csv", index=False)
    df_usgs_dash.to_csv("usgs_dashboard_sample.csv", index=False)
    
    print(f"✅ Données exportées:")
    print(f"   NOAA: {len(df_noaa_dash)} lignes")
    print(f"   USGS: {len(df_usgs_dash)} lignes")
    
    # Aperçu
    print("\n📋 Aperçu données dashboard:")
    print(df_noaa_dash.head())
else:
    print("ℹ️ Données insuffisantes pour l'export")

# %% [markdown]
# ### 10.2 Génération de rapports

# %%
print("📈 Génération de rapports...")

# Créer un rapport simple
report = {
    "generation_date": datetime.now().isoformat(),
    "noaa_stats": {
        "total_records": df_noaa.count() if df_noaa else 0,
        "date_range": "N/A" if df_noaa.count() == 0 else f"{df_noaa.agg({'date': 'min'}).collect()[0][0]} to {df_noaa.agg({'date': 'max'}).collect()[0][0]}",
        "stations_count": df_noaa.select("station_id").distinct().count() if df_noaa else 0
    },
    "usgs_stats": {
        "total_earthquakes": df_usgs.count() if df_usgs else 0,
        "avg_magnitude": df_usgs.agg({"magnitude": "avg"}).collect()[0][0] if df_usgs and df_usgs.count() > 0 else 0,
        "max_magnitude": df_usgs.agg({"magnitude": "max"}).collect()[0][0] if df_usgs and df_usgs.count() > 0 else 0
    },
    "data_quality": {
        "noaa_completeness": "N/A",
        "usgs_completeness": "N/A",
        "hdfs_connection": "✅" if spark._jsc is not None else "❌"
    }
}

print("\n📋 RAPPORT DATA LAKE")
print("=" * 50)
print(f"📅 Date: {report['generation_date']}")
print(f"📊 NOAA: {report['noaa_stats']['total_records']} enregistrements")
print(f"🌋 USGS: {report['usgs_stats']['total_earthquakes']} séismes")
print(f"🔗 Connexion HDFS: {report['data_quality']['hdfs_connection']}")
print("=" * 50)

# Sauvegarder le rapport
import json
with open("data_lake_report.json", "w") as f:
    json.dump(report, f, indent=2)

print("\n💾 Rapport sauvegardé: data_lake_report.json")

# %% [markdown]
# ## 11. Nettoyage et conclusion

# %%
# Arrêt de Spark
spark.stop()
print("\n👋 Spark arrêté")
print("✅ Notebook exécuté avec succès")